# 1 download J matrix and sequences


In [1]:
import os

url_j = "https://github.com/Daniel-Xkan/potts_model_test/raw/main/J.npy"
url_seq = "https://github.com/Daniel-Xkan/potts_model_test/raw/main/in.reduce4.seq"

# Suppress output and check for success
if os.system(f"wget -q {url_j}") == 0:
    print(f"Successfully downloaded {url_j}")
else:
    print(f"Failed to download {url_j}")

if os.system(f"wget -q {url_seq}") == 0:
    print(f"Successfully downloaded {url_seq}")
else:
    print(f"Failed to download {url_seq}")

Failed to download https://github.com/Daniel-Xkan/potts_model_test/raw/main/J.npy
Failed to download https://github.com/Daniel-Xkan/potts_model_test/raw/main/in.reduce4.seq


sh: wget: command not found
sh: wget: command not found


# 2. load and initialize the J matrix dictionary


In [2]:
import numpy as np

#dictionary of J matrix
J_dict = {}

# Load the J matrix from the downloaded file
J = np.load('J.npy')

row = 0
for pos1 in range(1, 264):
    for pos2 in range(pos1 + 1, 264):

        for i, aa1 in enumerate(['A', 'B', 'C', 'D']):
            for j, aa2 in enumerate(['A', 'B', 'C', 'D']):
                col = i*4+j


                J_dict[(pos1, pos2, aa1, aa2)] = J[row, col]
                J_dict[(pos2, pos1, aa2, aa1)] = J[row, col]
            # print(f"Row {row}: J[{row}, {col}] = {J[row, col]} for positions ({pos1}, {pos2}) with amino acids ({aa1}, {aa2})")
        row += 1

print(f"Dictionary created with {len(J_dict)} entries")

Dictionary created with 1102496 entries


# 3. load sequences

In [3]:
with open('in.reduce4.seq', 'r') as f:
    sequence = f.read().strip()
sequence_list = sequence.split('\n')

In [4]:
Consensus = sequence_list[0]
print(sequence_list[0])

ABAAABABACBBAACBDBABBDBDBBBAAAAAABAAABDAABAABAAABBBAAAAACACBAABAADACCBADAACACABBABABAAABABBAABDAABCABAABABAAAABDCBCAAADABCCADDAAAAAAAACDBDADBADAAABBABAAAAACBBADAAACCAAABAAABAAABAABABAABABABAAAACCAAAAAAAABBBBCACDCAACDBDCBCAAAAABADDBCACAAAAABAABAAAAAAABBDBDBACCBBBA


# 4. calculate delta delta E

## 4.1 define double delta E energy

In [5]:
def double_mutation_delta_delta_E(seq, pos1, aa1_new, pos2, aa2_new, J_dict, old_amino_acid1, old_amino_acid2):
    """
    Calculate delta delta E for a double mutation:
    delta_delta_E = delta_E_double - (delta_E_single1 + delta_E_single2)
    seq: original sequence (string)
    pos1, pos2: 1-based positions to mutate
    aa1_new, aa2_new: new amino acids at pos1 and pos2
    J_dict: dictionary of J matrix
    old_amino_acid1, old_amino_acid2: original amino acids at pos1 and pos2
    """
    # Single mutation at pos1
    delta_E_single1 = 0
    for other_pos in range(1, len(seq) + 1):
        if other_pos == pos1:
            continue
        other_aa = seq[other_pos - 1]
        delta_E_single1 += J_dict.get((pos1, other_pos, old_amino_acid1, other_aa), 0) - J_dict.get((pos1, other_pos, aa1_new, other_aa), 0)

    # Single mutation at pos2
    delta_E_single2 = 0
    for other_pos in range(1, len(seq) + 1):
        if other_pos == pos2:
            continue
        other_aa = seq[other_pos - 1]
        delta_E_single2 += J_dict.get((pos2, other_pos, old_amino_acid2, other_aa), 0) - J_dict.get((pos2, other_pos, aa2_new, other_aa), 0)

    # Double mutation
    seq_double = list(seq)
    seq_double[pos1 - 1] = aa1_new
    seq_double[pos2 - 1] = aa2_new

    delta_E_double = 0
    # Contribution for pos1
    for other_pos in range(1, len(seq) + 1):
        if other_pos == pos1:
            continue
        other_aa = seq_double[other_pos - 1]
        delta_E_double += J_dict.get((pos1, other_pos, aa1_new, other_aa), 0) - J_dict.get((pos1, other_pos, old_amino_acid1, seq[other_pos - 1]), 0)
    # Contribution for pos2
    for other_pos in range(1, len(seq) + 1):
        if other_pos == pos2:
            continue
        other_aa = seq_double[other_pos - 1]
        delta_E_double += J_dict.get((pos2, other_pos, aa2_new, other_aa), 0) - J_dict.get((pos2, other_pos, old_amino_acid2, seq[other_pos - 1]), 0)

    # Remove double-counted interaction between pos1 and pos2
    # (since both loops above include it twice)
    # aa1_old = seq[pos1 - 1]
    # aa2_old = seq[pos2 - 1]
    double_old = J_dict.get((pos1, pos2, old_amino_acid1, old_amino_acid2), 0)
    double_new = J_dict.get((pos1, pos2, aa1_new, aa2_new), 0)
    delta_E_double -= double_new - double_old

    delta_delta_E = delta_E_double - (delta_E_single1 + delta_E_single2)
    # print(delta_delta_E, delta_E_double,delta_E_single1, delta_E_single2)
    return delta_delta_E

## 4.2 define mutations

In [6]:
import itertools
import csv
from tqdm.notebook import tqdm # Import tqdm

output_file = 'delta_delta_E.tsv'

with open(output_file, 'w', newline='') as tsvfile:
    writer = csv.writer(tsvfile, delimiter='\t')
    writer.writerow(['mutation 1', 'mutation 2', 'delta delta E'])

    amino_acids = ['A', 'B', 'C', 'D']
    sequence_length = len(Consensus)

    # Calculate the total number of iterations for the progress bar
    total_iterations = 0
    for pos1 in range(1, sequence_length + 1):
        for pos2 in range(pos1 + 1, sequence_length + 1):
            old_amino_acid1 = Consensus[pos1 - 1]
            old_amino_acid2 = Consensus[pos2 - 1]
            for aa1_new in amino_acids:
                for aa2_new in amino_acids:
                    if aa1_new != old_amino_acid1 and aa2_new != old_amino_acid2:
                        total_iterations += 1

    # Iterate through all possible pairs of positions with a progress bar
    with tqdm(total=total_iterations, desc="Calculating Delta Delta E") as pbar:
        for pos1 in range(1, sequence_length + 1):
            for pos2 in range(pos1 + 1, sequence_length + 1):
                old_amino_acid1 = Consensus[pos1 - 1]
                old_amino_acid2 = Consensus[pos2 - 1]

                # Iterate through all possible new amino acids for pos1 and pos2
                for aa1_new in amino_acids:
                    for aa2_new in amino_acids:
                        # Only calculate if at least one amino acid is different
                        if aa1_new != old_amino_acid1 and aa2_new != old_amino_acid2:
                            dde = double_mutation_delta_delta_E(Consensus, pos1, aa1_new, pos2, aa2_new, J_dict, old_amino_acid1, old_amino_acid2)
                            mutation1_str = f"{old_amino_acid1}{pos1}{aa1_new}"
                            mutation2_str = f"{old_amino_acid2}{pos2}{aa2_new}"
                            writer.writerow([mutation1_str, mutation2_str, dde])
                            pbar.update(1) # Update the progress bar

print(f"Delta delta E values written to {output_file}")

ModuleNotFoundError: No module named 'tqdm'

In [1]:
import pandas as pd

# Load the delta delta E data
df = pd.read_csv('delta_delta_E.tsv', sep='\t')

# Define a threshold for strong interactions (example threshold, you might want to adjust this)
# For example, let's consider interactions with absolute delta delta E > 40 as strong
threshold = 40
filtered_df = df[abs(df['delta delta E']) > threshold].copy()

print(f"Original number of interactions: {len(df)}")
print(f"Number of strong interactions (abs(DDE) > {threshold}): {len(filtered_df)}")

display(filtered_df.head())

Original number of interactions: 310077
Number of strong interactions (abs(DDE) > 40): 17699


,mutation 1,mutation 2,delta delta E
11,A1B,A3D,40.748543
14,A1C,A3D,40.505570
17,A1D,A3D,40.805126
181,A1B,D22B,44.083984
184,A1C,D22B,43.840477
